In [1]:
import pandas as pd
import numpy as np
from readFile import split_into_values, toRPdata

In [2]:
# columns 와 value는 사용자 입력
df = pd.read_csv('resources/AXISX_resample.csv')
#df = pd.read_csv('resources/CLAMP_resample.csv')
value = ['value']
#df = pd.read_csv('resources/Dataset1.csv')
columns = ['chip', 'wire', 'segment']
value = ['value']

In [3]:
df = df.loc[:, columns + value] #('chip', 'wire', 'value')는 사용자 입력
size = 28 # 모델에 따라 변경 28, 96
result = split_into_values(df, columns)

In [4]:
# dataframe -> list
result_list = result.values.tolist()

In [5]:
from sklearn.preprocessing import MinMaxScaler

def MinMax(data):
    MMS = MinMaxScaler().fit(data)
    scaled = MMS.transform(data)
    return scaled

In [6]:
# result_list transpose
result_T = [list(x) for x in zip(*result_list)]

In [7]:
# minmax 정규화
result_scaled = MinMax(result_T)

In [8]:
# 다시 result transpose 해서 원래대로
result_scaled = [list(x) for x in zip(*result_scaled)]

In [9]:
from tslearn.preprocessing import TimeSeriesScalerMeanVariance, TimeSeriesResampler

In [10]:
# 2. 시계열 셋 크기 변경
result_ = TimeSeriesResampler(sz=size).fit_transform(result)

In [11]:
data = result_.reshape(result_.shape[0], 1, size)
X = toRPdata(data, threshold='point', percentage=30) # rp 1
#X = toRPdata(data) # rp 2

In [12]:
X_scaled = np.expand_dims(X, axis=3)
X_scaled.shape

(1330, 28, 28, 1)

In [13]:
from keras.models import load_model
model = load_model('AXISX28_weight.h5')

In [14]:
print(model.summary())

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_50 (Conv2D)           (None, 28, 28, 16)        160       
_________________________________________________________________
activation_50 (Activation)   (None, 28, 28, 16)        0         
_________________________________________________________________
max_pooling2d_20 (MaxPooling (None, 14, 14, 16)        0         
_________________________________________________________________
conv2d_51 (Conv2D)           (None, 14, 14, 2)         290       
_________________________________________________________________
activation_51 (Activation)   (None, 14, 14, 2)         0         
_________________________________________________________________
max_pooling2d_21 (MaxPooling (None, 7, 7, 2)           0         
_________________________________________________________________
conv2d_52 (Conv2D)           (None, 7, 7, 2)         

In [15]:
from keras import backend as K

compressed_layer = 5
get_layer_output = K.function([model.layers[0].input],[model.layers[compressed_layer].output])
compressed = get_layer_output([X_scaled])[0]
feature = compressed.reshape(compressed.shape[0], compressed.shape[1]*compressed.shape[2]*compressed.shape[3])

print(feature)
print(feature.shape)

[[ 2.8124526   9.155897    2.4996412  ...  4.843092    0.97782266
   5.9397783 ]
 [ 0.72931063  9.4088745   1.231954   ...  9.288801    0.
   8.6982765 ]
 [ 3.5434444  13.811562    3.0308454  ...  8.052279    0.09402312
  11.31831   ]
 ...
 [ 1.1788112  11.487696    3.5470057  ...  3.2053933   0.09765781
   5.9569764 ]
 [ 0.87629616  9.354071    1.0341865  ...  3.5892596   2.9355578
   8.77942   ]
 [ 0.32359755  8.814086    2.4931443  ...  6.991415    3.5413141
   8.522947  ]]
(1330, 98)


In [16]:
import numpy

numpy.save('./compressed4', compressed) # x_save.npy
numpy.save('./feature4', feature) # x_save.npy
numpy.save('./X_scaled4', X_scaled) # x_save.npy